# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 2: Implementacion de heurísticas para el Problema de Busqueda de Rutas

La tarea de este laboratorio consiste en implementar y comparar funciones heurísticas para A* en el problema de busqueda de rutas en mapas. La clase MapSearchProblem, implementada en el laboratorio 1, tendra que ser modificada/ampliada para soportar mapas con localizaciones de las ciudades (coordenadas X,Y). Abajo puede encontrar la definición de la clase Mapa y una instancia específica con el mapa de Romania que sera usado en las pruebas. Mapa tiene dos atributos: <b>neighbors</b> (diccionario que contiene las ciudades vecinas de cada ciudad y las distancias para llegar a ellas, igual que el Laboratorio 1) y <b>location</b>, diccionario con las coordenadas X,Y de cada ciudad. 

Mas abajo estan las definiciones de la clase Problem, MapSearchProblem, Node, FrontierPQ (una frontera tipo cola de prioridad ordenada por una funcion especificada f, para best_first_graph_search y A*), best_first_graph_search y astar_search (A*). 

Al final de este notebook estan las tareas y preguntas que seran evaluadas en este laboratorio. 


<b>Mapa de Romania con localizaciones para pruebas</b>

In [14]:
class Mapa:
    neighbors = dict()
    location = dict()
    
romania = Mapa()
romania.neighbors = {
 'A': [('Z',75), ('T',118), ('S',140)],
 'B': [('F',211), ('P',101), ('G',90), ('U',85)],
 'C': [('D',120), ('R',146), ('P',138)],
 'D': [('M',75), ('C',120)],
 'E': [('H',86)],
 'F': [('S',99), ('B',211)],
 'G': [('B',90)],
 'H': [('U',98), ('E',86)],
 'I': [('N',87), ('V',92)],
 'L': [('T',111), ('M',70)],
 'M': [('L',70), ('D',75)],
 'N': [('I',87)],
 'O': [('Z',71), ('S',151)],
 'P': [('R',97), ('C',138), ('B',101)],
 'R': [('S',80), ('C',146), ('P',97)],
 'S': [('A',140), ('O',151), ('F',99), ('R',80)],
 'T': [('A',118), ('L',111)],
 'U': [('B',85), ('V',142), ('H',98)],
 'V': [('U',142), ('I',92)],
 'Z': [('O',71), ('A',75)]}
romania.location = {
 'A': (91, 492),
 'B': (400, 327),
 'C': (253, 288),
 'D': (165, 299),
 'E': (562, 293),
 'F': (305, 449),
 'G': (375, 270),
 'H': (534, 350),
 'I': (473, 506),
 'L': (165, 379),
 'M': (168, 339),
 'N': (406, 537),
 'O': (131, 571),
 'P': (320, 368),
 'R': (233, 410),
 'S': (207, 457),
 'T': (94, 410),
 'U': (456, 350),
 'V': (509, 444),
 'Z': (108, 531)}

<b> Clase genérica Problem (no modificar) </b>

In [15]:
class Problem(object):
    def __init__(self, initial, goal=None):
        """Este constructor especifica el estado inicial y posiblemente el estado(s) objetivo(s),
        La subclase puede añadir mas argumentos."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Retorna las acciones que pueden ser ejecutadas en el estado dado.
        El resultado es tipicamente una lista."""
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada en el estado state.
        La accion debe ser alguna de self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Retorna True si el estado pasado satisface el objetivo."""
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con 
        la accion action, asumiendo un costo c para llegar hasta state1. 
        El metodo por defecto cuesta 1 para cada paso en el camino."""
        return c + 1

    def value(self, state):
        """En problemas de optimizacion, cada estado tiene un valor. Algoritmos
        como Hill-climbing intentan maximizar este valor."""
        raise NotImplementedError

<b> Clase MapSearchProblem </b>  (define el problema de busqueda en mapa)

In [16]:
class MapSearchProblem(Problem):
    def __init__(self, initial, goal, mapa):
        """El constructor recibe  el estado inicial, el estado objetivo y un mapa (de clase Mapa)"""
        self.initial = initial
        self.goal = goal
        self.map = mapa

    def actions(self, state):
        """Retorna las acciones ejecutables desde ciudad state.
        El resultado es una lista de strings tipo 'goCity'. 
        Por ejemplo, en el mapa de Romania, las acciones desde Arad serian:
         ['goZerind', 'goTimisoara', 'goSibiu']"""
        #TODO
        ciudadesCercanas= self.map.neighbors[state]
        self.actio=['go'+ i for i,j in ciudadesCercanas]  
        return self.actio
        raise NotImplementedError

    def result(self, state, action):
        """Retorna el estado que resulta de ejecutar la accion dada desde ciudad state.
        La accion debe ser alguna de self.actions(state)
        Por ejemplo, en el mapa de Romania, el resultado de aplicar la accion 'goZerind' 
        desde el estado 'Arad' seria 'Zerind'"""  
        #TODO
        for act in self.actions(state):
            if act==action:
                self.state=act[2:]  
        return self.state        
        raise NotImplementedError
        
    def goal_test(self, state):
        """Retorna True si state es self.goal"""
        #TODO
        return state==self.goal
        raise NotImplementedError

    def path_cost(self, c, state1, action, state2):
        """Retorna el costo del camino de state2 viniendo de state1 con la accion action 
        El costo del camino para llegar a state1 es c. El costo de la accion debe ser
        extraido de self.map."""
        #TODO
        costo=0
        ciudadesCercanas= self.map.neighbors[state1]
        estadosPosibles=[i for i,j in ciudadesCercanas]
        costosPosibles= [j for i,j in ciudadesCercanas]
        a=0
        for estado in estadosPosibles:
            if estado==state2:
                costo=costosPosibles[a]
            a=a+1                    
        self.cost=costo+c
        return self.cost
        raise NotImplementedError

<b> Clase Node </b>

In [17]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        "Crea un nodo de arbol de busqueda, derivado del nodo parent y accion action"
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        "Devuelve los nodos alcanzables en un paso a partir de este nodo."
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next = problem.result(self.state, action)
        return Node(next, self, action,
                    problem.path_cost(self.path_cost, self.state, action, next))

    def solution(self):
        "Retorna la secuencia de acciones para ir de la raiz a este nodo."
        return [node.action for node in self.path()[1:]]

    def path(self):
        "Retorna una lista de nodos formando un camino de la raiz a este nodo."
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def __eq__(self, other): 
        "Este metodo se ejecuta cuando se compara nodos. Devuelve True cuando los estados son iguales"
        return isinstance(other, Node) and self.state == other.state

<b> Frontera tipo cola de prioridad ordenada por una funcion de costo (para best_first_graph_search y A*)</b> 

In [18]:
import heapq
class FrontierPQ:
    "Una Frontera ordenada por una funcion de costo (Priority Queue)"
    
    def __init__(self, initial, costfn=lambda node: node.path_cost):
        "Inicializa la Frontera con un nodo inicial y una funcion de costo especificada (por defecto es el costo de camino)."
        self.heap   = []
        self.states = {}
        self.costfn = costfn
        self.add(initial)
    
    def add(self, node):
        "Agrega un nodo a la frontera."
        cost = self.costfn(node)
        heapq.heappush(self.heap, (cost, node))
        self.states[node.state] = node
        
    def pop(self):
        "Remueve y retorna el nodo con minimo costo."
        (cost, node) = heapq.heappop(self.heap)
        self.states.pop(node.state, None) # remove state
        return node
    
    def replace(self, node):
        "node reemplaza al nodo de la Fontera que tiene el mismo estado que node."
        if node.state not in self:
            raise ValueError('{} no tiene nada que reemplazar'.format(node.state))
        for (i, (cost, old_node)) in enumerate(self.heap):
            if old_node.state == node.state:
                self.heap[i] = (self.costfn(node), node)
                heapq._siftdown(self.heap, 0, i)
                return

    def __contains__(self, state): return state in self.states
    
    def __len__(self): return len(self.heap)

<b> Algoritmo Best-First-Graph-Search </b> 

In [19]:
def best_first_graph_search(problem, f):
    """Busca el objetivo expandiendo el nodo de la frontera con el menor valor de la funcion f. Memoriza estados visitados
    Antes de llamar a este algoritmo hay que especificar La funcion f(node). Si f es node.depth tenemos Busqueda en Amplitud; 
    si f es node.path_cost tenemos Busqueda  de Costo Uniforme. Si f es una heurística tenemos Busqueda Voraz;
    Si f es node.path_cost + heuristica(node) tenemos A* """

    frontier = FrontierPQ( Node(problem.initial), f )  # frontera tipo cola de prioridad ordenada por f
    explored = set()     # memoria de estados visitados
    expanded_nodes = 0   # contador de nodos expandidos
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node, expanded_nodes
        explored.add(node.state)
        expanded_nodes = expanded_nodes + 1
        for action in problem.actions(node.state):
            child = node.child_node(problem, action)
            if child.state not in explored and child.state not in frontier:
                frontier.add(child)
            elif child.state in frontier:
                incumbent = frontier.states[child.state] 
                if f(child) < f(incumbent):
                    frontier.replace(child)

<b> Algoritmo A* </b> 

In [20]:
def astar_search(problem, heuristic):
    f = lambda node: node.path_cost + heuristic(node, problem)
    return best_first_graph_search(problem, f)

## Tareas y Cuestiones

<b>1) Implementar las dos heurísticas abajo para A*: </b>
  

In [25]:
import math 
def h1(node, problem):
    "Distancia en linea recta desde la posicion de node hasta la posicion del Objetivo (problem.goal)"
    #TODO
    #Conseguimos la locacion del nodo dado
    (xa,ya)=problem.map.location[node.state]
    #Conseguimos la locacion del nodo objetivo
    (xb,yb)=problem.map.location[problem.goal]
    
    #Hallamos la distancia en línea recta
    d= math.sqrt((xa-xb)**2+(ya-yb)**2)
    return d

def h2(node, problem):
    "Distancia Manhatan (o city block) desde la posicion de node hasta la posicion del Objetivo (problem.goal)"
    #TODO
    #Conseguimos la locacion del nodo dado
    (xa,ya)=problem.map.location[node.state]
    #Conseguimos la locacion del nodo objetivo
    (xb,yb)=problem.map.location[problem.goal]
    
    #Hallamos la distancia Manhatan
    d= abs(xa-xb)+abs(ya-yb)
    return d

<b>2) Ejecutar la busqueda A* con ambas heurísticas en los siguientes problemas:  (hasta 10 puntos si ejecuta correctamente)</b>

In [26]:
p1 = MapSearchProblem('A', 'B', romania)
p2 = MapSearchProblem('O', 'N', romania)
p3 = MapSearchProblem('T', 'V', romania)

node, num_exp_nodes = astar_search(p1, h1)
print( 'Solucion del Problema p1 con heuristica h1: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = astar_search(p1, h2)
print( 'Solucion del Problema p1 con heuristica h2: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = astar_search(p2, h1)
print( 'Solucion del Problema p2 con heuristica h1: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = astar_search(p2, h2)
print( 'Solucion del Problema p2 con heuristica h2: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = astar_search(p3, h1)
print( 'Solucion del Problema p3 con heuristica h1: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )

node, num_exp_nodes = astar_search(p3, h2)
print( 'Solucion del Problema p3 con heuristica h2: {}. Nodos expandidos = {}'.format(node.solution(), num_exp_nodes) )


Solucion del Problema p1 con heuristica h1: ['goS', 'goR', 'goP', 'goB']. Nodos expandidos = 5
Solucion del Problema p1 con heuristica h2: ['goS', 'goF', 'goB']. Nodos expandidos = 3
Solucion del Problema p2 con heuristica h1: ['goS', 'goR', 'goP', 'goB', 'goU', 'goV', 'goI', 'goN']. Nodos expandidos = 16
Solucion del Problema p2 con heuristica h2: ['goS', 'goR', 'goP', 'goB', 'goU', 'goV', 'goI', 'goN']. Nodos expandidos = 15
Solucion del Problema p3 con heuristica h1: ['goA', 'goS', 'goR', 'goP', 'goB', 'goU', 'goV']. Nodos expandidos = 14
Solucion del Problema p3 con heuristica h2: ['goA', 'goS', 'goR', 'goP', 'goB', 'goU', 'goV']. Nodos expandidos = 12


<b>3) Basados en los resultados anteriores responda: (10 puntos)</b>

3.1) ¿Cual heurística tiende a encontrar la solucion con menos nodos expandidos? Porque?

La heurística 2 tiende a encontrar la solución con menos nodos expandidos, porque como sigue una distancia mayor a la que se consigue mediante la línea recta, se desplaza por menos nodos pero con posiblemente mayor costo de camino.

3.2) ¿Las dos heurísticas encuentran siempre la solución optima? Porque?

No siempre. En el ejemplo que se va de la ciudad 'A' a la ciudad 'B' la solución óptima de costo de camino es la que traza la heurística 1, mientras que la 2 logra visitar menos nodos teniendo un mayor costo de camino.

El motivo por el cual se cumple esto es que la heurística 2 no es admisible (porque sobreestima el costo de alcanzar el objetivo) y su función no minimiza el costo de camino tan bien como lo puede hacer la heurística 1. 


